<a href="https://colab.research.google.com/github/theekshiragula092/GitHUB/blob/main/sales_dashboard_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-darkgrid')

# Class 1: SalesDataLoader
# SOLID Principle: Single Responsibility - This class is solely responsible for loading and initially cleaning the sales data.
class SalesDataLoader:
    def __init__(self, file_path):
        self.file_path = file_path # Data Structure: String to hold the file path
        self.data = None # Data Structure: Placeholder for the pandas DataFrame

    def load_data(self):
        try:
            # Data Structure: Reading data into a pandas DataFrame
            self.data = pd.read_csv(self.file_path)

            # Convert 'Date' column to datetime objects
            # Data Structure: Converting a column to datetime format
            self.data['Date'] = pd.to_datetime(self.data['Date'])

            # Clean and convert 'Price (LKR)' and 'Total Amount (LKR)' to numeric
            # Clean Code: Data cleaning steps are encapsulated within the loading process.
            self.data['Price (LKR)'] = self.data['Price (LKR)'].astype(str).str.replace('LKR ', '').str.replace(',', '').astype(float)
            self.data['Total Amount (LKR)'] = self.data['Total Amount (LKR)'].astype(str).str.replace('LKR ', '').str.replace(',', '').astype(float)

            # Ensure 'Quantity' is numeric
            self.data['Quantity'] = pd.to_numeric(self.data['Quantity'], errors='coerce')

            # Drop rows with NaN in critical columns
            # Clean Code: Handling missing data
            self.data.dropna(subset=['Total Amount (LKR)', 'Price (LKR)', 'Quantity'], inplace=True)

            print("Data loaded successfully.")
            return self.data # Data Structure: Returning the processed DataFrame
        except FileNotFoundError:
            print(f"Error: The file at {self.file_path} was not found.")
            return None
        except Exception as e:
            print(f"An error occurred while loading data: {e}")
            return None

# Class 2: MonthlySalesAnalysis
# SOLID Principle: Single Responsibility - This class is responsible for analyzing and visualizing monthly sales by branch.
class MonthlySalesAnalysis:
    def __init__(self, data):
        # Data Structure: Working on a copy of the input DataFrame to avoid modifying the original
        self.data = data.copy()

    def analyze(self):
        if self.data is None or self.data.empty:
            print("No data available for MonthlySalesAnalysis.")
            return

        print("\n--- MonthlySalesAnalysis: Monthly Sales by Branch ---")
        # Data Structure: Grouping and aggregating data using pandas
        monthly_sales = self.data.groupby('Branch')['Total Amount (LKR)'].sum().sort_values(ascending=False)

        if monthly_sales.empty:
            print("No sales data to display for branches.")
            return

        # Clean Code: Using matplotlib for visualization, separated from data processing
        fig, ax = plt.subplots(figsize=(10, 6))
        colors = plt.colormaps.get_cmap('Paired') # Clean Code: Using a colormap for visual appeal
        monthly_sales.plot(kind='bar', ax=ax, color=colors(range(len(monthly_sales))))

        # Clean Code: Clear and descriptive plot labels and title
        ax.set_title('Monthly Sales by Branch', fontsize=16)
        ax.set_xlabel('Branch', fontsize=12)
        ax.set_ylabel('Sales (LKR)', fontsize=12)
        ax.ticklabel_format(style='plain', axis='y')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

# Class 3: ProductPriceAnalysis
# SOLID Principle: Single Responsibility - This class focuses on analyzing and visualizing product price trends over months.
class ProductPriceAnalysis:
    def __init__(self, data):
        # Data Structure: Working on a copy of the input DataFrame
        self.data = data.copy()

    def analyze(self):
        if self.data is None or self.data.empty:
            print("No data available for ProductPriceAnalysis.")
            return

        print("\n--- ProductPriceAnalysis: Product Price Over Months ---")
        # Data Structure: Creating a 'Month' period column
        self.data['Month'] = self.data['Date'].dt.to_period('M')
        # Data Structure: Grouping, aggregating, and reshaping data using pandas (pivot-like operation)
        price_trends = self.data.groupby(['Product', 'Month'])['Price (LKR)'].mean().unstack(fill_value=0)

        if price_trends.empty:
            print("No product price data to display.")
            return

        price_trends = price_trends.reindex(sorted(price_trends.columns), axis=1)
        price_trends.columns = price_trends.columns.astype(str)

        # Clean Code: Using matplotlib for visualization
        fig, ax = plt.subplots(figsize=(14, 8))
        for product in price_trends.index:
            # Clean Code: Iterating through products for individual trend lines
            ax.plot(price_trends.columns, price_trends.loc[product], marker='o', label=product)

        # Clean Code: Clear and descriptive plot labels and title
        ax.set_title('Average Product Price Trend Over Months', fontsize=16)
        ax.set_xlabel('Month', fontsize=12)
        ax.set_ylabel('Average Unit Price (LKR)', fontsize=12)
        ax.ticklabel_format(style='plain', axis='y')
        ax.legend(title='Product', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small') # Clean Code: Placing legend outside plot to avoid overlap
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

# Class 4: WeeklySalesAnalysis
# SOLID Principle: Single Responsibility - This class is dedicated to analyzing and visualizing weekly sales trends.
class WeeklySalesAnalysis:
    def __init__(self, data):
        # Data Structure: Working on a copy of the input DataFrame
        self.data = data.copy()

    def analyze(self):
        if self.data is None or self.data.empty:
            print("No data available for WeeklySalesAnalysis.")
            return

        print("\n--- WeeklySalesAnalysis: Weekly Sales Across Supermarket Network ---")
        # Data Structure: Extracting week and year from the Date column
        self.data['Week'] = self.data['Date'].dt.isocalendar().week.astype(int)
        self.data['Year'] = self.data['Date'].dt.year
        # Data Structure: Grouping and aggregating weekly sales
        weekly_sales = self.data.groupby(['Year', 'Week'])['Total Amount (LKR)'].sum().reset_index()
        weekly_sales['Year_Week'] = weekly_sales['Year'].astype(str) + '-W' + weekly_sales['Week'].astype(str).str.zfill(2)
        weekly_sales.sort_values(by=['Year', 'Week'], inplace=True) # Clean Code: Sorting data for chronological plotting

        if weekly_sales.empty:
            print("No weekly sales data to display.")
            return

        # Clean Code: Using matplotlib for visualization
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(weekly_sales['Year_Week'], weekly_sales['Total Amount (LKR)'], marker='o', linestyle='-', color='teal') # Clean Code: Using a specific color for the line plot

        # Clean Code: Clear and descriptive plot labels and title
        ax.set_title('Weekly Sales Across Supermarket Network', fontsize=16)
        ax.set_xlabel('Week', fontsize=12)
        ax.set_ylabel('Total Sales (LKR)', fontsize=12)
        ax.ticklabel_format(style='plain', axis='y')
        n_weeks = len(weekly_sales)
        if n_weeks > 15:
            ax.set_xticks(weekly_sales['Year_Week'][::n_weeks // 10 + 1]) # Clean Code: Adjusting tick frequency for readability
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

# Class 5: ProductPreferenceAnalysis
# SOLID Principle: Single Responsibility - This class is for analyzing and visualizing product purchase preferences.
class ProductPreferenceAnalysis:
    def __init__(self, data):
        # Data Structure: Working on a copy of the input DataFrame
        self.data = data.copy()

    def analyze(self, selected_branch=None):
        if self.data is None or self.data.empty:
            print("No data available for ProductPreferenceAnalysis.")
            return

        print("\n--- ProductPreferenceAnalysis: Product Preference Analysis ---")
        filtered_data = self.data # Data Structure: Creating a filtered DataFrame based on branch
        if selected_branch:
            print(f"Analyzing product preferences for: {selected_branch}")
            filtered_data = self.data[self.data['Branch'] == selected_branch]
            if filtered_data.empty:
                print(f"No data for branch: {selected_branch}")
                return

        # Data Structure: Grouping and summing quantity to find preferences
        product_purchase_counts = filtered_data.groupby('Product')['Quantity'].sum().sort_values(ascending=False)

        if product_purchase_counts.empty:
            print("No product preference data to display.")
            return

        # Clean Code: Using matplotlib for visualization
        fig, ax = plt.subplots(figsize=(12, min(len(product_purchase_counts) * 0.4, 10))) # Clean Code: Adjusting figure size dynamically
        colors = plt.colormaps.get_cmap('viridis') # Clean Code: Using a colormap for visual appeal
        product_purchase_counts.plot(kind='barh', ax=ax, color=colors(range(len(product_purchase_counts)))) # Clean Code: Using horizontal bar plot for readability

        # Clean Code: Clear and descriptive plot labels and title
        ax.set_title(f'Top Product Preferences (Total Units Purchased){f" for {selected_branch}" if selected_branch else ""}', fontsize=16)
        ax.set_xlabel('Total Units Purchased', fontsize=12)
        ax.set_ylabel('Product Name', fontsize=12)
        plt.gca().invert_yaxis() # Clean Code: Inverting y-axis for better ranking visualization
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

# Class 6: TotalSalesDistributionAnalysis
# SOLID Principle: Single Responsibility - This class focuses on visualizing the distribution of total sales by branch using a pie chart.
class TotalSalesDistributionAnalysis:
    def __init__(self, data):
        # Data Structure: Working on a copy of the input DataFrame
        self.data = data.copy()

    def analyze(self):
        if self.data is None or self.data.empty:
            print("No data available for TotalSalesDistributionAnalysis.")
            return

        print("\n--- TotalSalesDistributionAnalysis: Total Sales Amount Distribution by Branch ---")
        # Data Structure: Grouping and summing total sales by branch
        branch_total_sales = self.data.groupby('Branch')['Total Amount (LKR)'].sum()

        if branch_total_sales.empty:
            print("No total sales distribution data to display.")
            return

        # Clean Code: Using matplotlib for visualization (pie chart)
        fig, ax = plt.subplots(figsize=(10, 8))
        colors_list = plt.colormaps.get_cmap('Set2').colors # Clean Code: Using a colormap for pie slices
        wedges, texts, autotexts = ax.pie(
            branch_total_sales,
            labels=branch_total_sales.index, # Clean Code: Using branch names as labels
            autopct='%1.1f%%', # Clean Code: Displaying percentage on slices
            startangle=90,
            colors=colors_list[:len(branch_total_sales)],
            pctdistance=0.85 # Clean Code: Adjusting percentage position
        )
        centre_circle = plt.Circle((0,0), 0.70, fc='white') # Clean Code: Creating a donut chart effect
        fig.gca().add_artist(centre_circle)

        # Clean Code: Clear and descriptive plot title
        ax.set_title('Distribution of Total Sales Amount by Branch', fontsize=16)
        ax.axis('equal') # Clean Code: Ensuring the pie chart is circular
        plt.tight_layout()
        plt.show()

# Main Dashboard Class
# Design Pattern: This class acts as a simple orchestrator or facade, bringing together the different analysis modules.
# SOLID Principle: Open/Closed Principle - New analysis classes can be added without modifying existing analysis classes.
# SOLID Principle: Dependency Inversion Principle - Depends on abstractions (analysis classes) rather than concrete implementations (though not strictly enforced with interfaces here).
class APDPDashboard:
    def __init__(self, file_path):
        self.data_loader = SalesDataLoader(file_path) # SOLID Principle: Dependency Injection - Injecting the data loader dependency
        self.data = None # Data Structure: Placeholder for the loaded data
        self.analyses = {} # Data Structure: Dictionary to hold different analysis objects
        self.available_branches = [] # Data Structure: List to hold available branches
        self.output = widgets.Output() # Data Structure: ipywidgets Output widget

    def load_and_initialize(self):
        # Clean Code: Separating initialization logic from the main run method
        self.data = self.data_loader.load_data()
        if self.data is not None:
            # Design Pattern: Instantiating different analysis modules
            self.analyses['MonthlySalesAnalysis'] = MonthlySalesAnalysis(self.data)
            self.analyses['ProductPriceAnalysis'] = ProductPriceAnalysis(self.data)
            self.analyses['WeeklySalesAnalysis'] = WeeklySalesAnalysis(self.data)
            self.analyses['ProductPreferenceAnalysis'] = ProductPreferenceAnalysis(self.data)
            self.analyses['TotalSalesDistributionAnalysis'] = TotalSalesDistributionAnalysis(self.data)
            # Data Structure: Populating the list of branches
            self.available_branches = sorted(self.data['Branch'].unique().tolist())
        else:
            print("Failed to load data.")


    # Clean Code: Using a dedicated method to handle button clicks
    def on_button_clicked(self, b):
        with self.output: # Clean Code: Using the output widget for controlled display
            clear_output(wait=True) # Clean Code: Clearing previous output
            choice = b.description # Clean Code: Using button description to identify the action
            if choice == 'Exit':
                print("Exiting Dashboard. Have a nice day, Goodbye!")
            elif choice == 'ShowAllAnalyses':
                # Clean Code: Iterating through analyses to display all
                for key in ['MonthlySalesAnalysis', 'ProductPriceAnalysis', 'WeeklySalesAnalysis', 'ProductPreferenceAnalysis', 'TotalSalesDistributionAnalysis']:
                    print(f"\n--- Running {key} ---")
                    if key == 'ProductPreferenceAnalysis':
                        print("\n--- Product Preference Analysis Options ---")
                        # Data Structure: Creating options for a dropdown menu
                        # Include "All Branches" option
                        branch_options_labels = self.available_branches + ["All Branches"]
                        branch_options_values = [str(i+1) for i in range(len(self.available_branches))] + [str(len(self.available_branches)+1)]
                        branch_options = list(zip(branch_options_labels, branch_options_values))

                        branch_menu = widgets.Dropdown( # Data Structure: ipywidgets Dropdown
                            options=branch_options,
                            description='Select Branch:',
                            layout={'width': 'initial'}
                        )
                        display(branch_menu)
                        # Clean Code: Defining an event handler for dropdown changes
                        def on_branch_change(change):
                            with self.output:
                                clear_output(wait=True)
                                branch_idx = int(change.new) - 1
                                if branch_idx == len(self.available_branches):
                                    # Handle "All Branches" selection
                                    self.analyses[key].analyze(selected_branch=None)
                                elif 0 <= branch_idx < len(self.available_branches):
                                    selected_branch = self.available_branches[branch_idx]
                                    self.analyses[key].analyze(selected_branch=selected_branch)
                                else:
                                    print("Invalid branch selection. Showing overall product preference.")
                                    self.analyses[key].analyze(selected_branch=None)
                        branch_menu.observe(on_branch_change, names='value') # Clean Code: Observing dropdown value changes

                        # Initially display the "All Branches" product preference analysis when "Show All Analyses" is clicked
                        self.analyses[key].analyze(selected_branch=None)

                    else:
                        self.analyses[key].analyze() # Clean Code: Calling analyze method of the selected analysis
                print("\nAll analyses displayed.")
            elif choice in self.analyses:
                # Clean Code: Handling individual analysis selection
                if choice == 'ProductPreferenceAnalysis':
                    print("\n--- Product Preference Analysis Options ---")
                    # Include "All Branches" option
                    branch_options_labels = self.available_branches + ["All Branches"]
                    branch_options_values = [str(i+1) for i in range(len(self.available_branches))] + [str(len(self.available_branches)+1)]
                    branch_options = list(zip(branch_options_labels, branch_options_values))

                    branch_menu = widgets.Dropdown(
                        options=branch_options,
                        description='Select Branch:',
                        layout={'width': 'initial'}
                    )
                    display(branch_menu)
                    def on_branch_change(change):
                        with self.output:
                            clear_output(wait=True)
                            branch_idx = int(change.new) - 1
                            if branch_idx == len(self.available_branches):
                                # Handle "All Branches" selection
                                self.analyses[choice].analyze(selected_branch=None)
                            elif 0 <= branch_idx < len(self.available_branches):
                                selected_branch = self.available_branches[branch_idx]
                                self.analyses[choice].analyze(selected_branch=selected_branch)
                            else:
                                print("Invalid branch selection. Showing overall product preference.")
                                self.analyses[choice].analyze(selected_branch=None)
                    branch_menu.observe(on_branch_change, names='value')

                    # Initially display the "All Branches" product preference analysis when the individual button is clicked
                    self.analyses[choice].analyze(selected_branch=None)

                else:
                    self.analyses[choice].analyze()
            else:
                print("Invalid choice. Please try again.") # Clean Code: User feedback for invalid input

    # Clean Code: Main method to run the dashboard interface
    def run(self):
        self.load_and_initialize() # Clean Code: Calling initialization method
        button_layout = widgets.Layout(width='200px', height='40px') # Clean Code: Consistent layout for buttons

        # Clean Code: Creating interactive buttons using ipywidgets
        button1 = widgets.Button(description='MonthlySalesAnalysis', layout=button_layout)
        button2 = widgets.Button(description='ProductPriceAnalysis', layout=button_layout)
        button3 = widgets.Button(description='WeeklySalesAnalysis', layout=button_layout)
        button4 = widgets.Button(description='ProductPreferenceAnalysis', layout=button_layout)
        button5 = widgets.Button(description='TotalSalesDistributionAnalysis', layout=button_layout)
        button6 = widgets.Button(description='ShowAllAnalyses', layout=button_layout)
        button0 = widgets.Button(description='Exit', layout=button_layout)

        # Clean Code: Attaching the click handler to each button
        button1.on_click(self.on_button_clicked)
        button2.on_click(self.on_button_clicked)
        button3.on_click(self.on_button_clicked)
        button4.on_click(self.on_button_clicked)
        button5.on_click(self.on_button_clicked)
        button6.on_click(self.on_button_clicked)
        button0.on_click(self.on_button_clicked)

        # Clean Code: Arranging buttons and output using ipywidgets layout
        button_box = widgets.VBox([button1, button2, button3, button4, button5, button6, button0])
        display(widgets.HBox([button_box, self.output])) # Clean Code: Displaying the UI
# --- Run the Dashboard ---
if __name__ == "__main__":
  csv_file_path = "/content/drive/MyDrive/Colab Notebooks/sampath_sales_data.csv"
  dashboard = APDPDashboard(csv_file_path)
  dashboard.run()

Data loaded successfully.


In [5]:
T = """\
import pytest
import pandas as pd
import nbformat
import sys
import os
import types
from IPython import get_ipython
from nbconvert import PythonExporter

# --- Dynamically import the notebook as a module ---
def import_ipynb_as_module(notebook_path, module_name="sales_dashboard_colab"):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    exporter = PythonExporter()
    source_code, _ = exporter.from_notebook_node(nb)

    # Remove the problematic shell command if it exists
    # The problematic line is "!pytest test_sales_dashboard.py" or "pytest test_sales_dashboard.py"
    problematic_lines = ["!pytest test_sales_dashboard.py", "pytest test_sales_dashboard.py"]
    lines = source_code.splitlines()
    filtered_lines = [line for line in lines if not any(prob_line in line for prob_line in problematic_lines)]
    source_code = "\\n".join(filtered_lines)


    # Print the generated source code for debugging
    print("--- Generated Source Code ---")
    print(source_code)
    print("---------------------------")

    module = types.ModuleType(module_name)
    exec(source_code, module.__dict__)
    sys.modules[module_name] = module
    return module

# Load your notebook as a module
notebook_path = "/content/drive/MyDrive/Colab Notebooks/sales_dashboard_colab.ipynb"

sales_dashboard_colab = import_ipynb_as_module(notebook_path)

# Import classes from the dynamically loaded module
SalesDataLoader = sales_dashboard_colab.SalesDataLoader
MonthlySalesAnalysis = sales_dashboard_colab.MonthlySalesAnalysis
WeeklySalesAnalysis = sales_dashboard_colab.WeeklySalesAnalysis
ProductPriceAnalysis = sales_dashboard_colab.ProductPriceAnalysis
ProductPreferenceAnalysis = sales_dashboard_colab.ProductPreferenceAnalysis
TotalSalesDistributionAnalysis = sales_dashboard_colab.TotalSalesDistributionAnalysis

# Path to your real CSV file
TEST_CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/sampath_sales_data.csv"

@pytest.fixture(scope="module")
def loaded_data():
    loader = SalesDataLoader(TEST_CSV_PATH)
    df = loader.load_data()
    assert df is not None
    return df

def test_monthly_sales_analysis(loaded_data):
    analysis = MonthlySalesAnalysis(loaded_data)
    # Assuming the analyze method processes and potentially returns something or modifies self.data
    # For testing purposes, we'll replicate the core logic here to assert on the result
    grouped_data = loaded_data.groupby('Branch')['Total Amount (LKR)'].sum()
    assert not grouped_data.empty
    assert grouped_data.dtype == float

def test_weekly_sales_analysis(loaded_data):
    analysis = WeeklySalesAnalysis(loaded_data)
    # Replicate core logic for testing
    loaded_data_copy = loaded_data.copy() # Work on a copy to not affect other tests
    loaded_data_copy['Week'] = loaded_data_copy['Date'].dt.isocalendar().week.astype(int)
    loaded_data_copy['Year'] = loaded_data_copy['Date'].dt.year
    weekly = loaded_data_copy.groupby(['Year', 'Week'])['Total Amount (LKR)'].sum().reset_index()
    weekly['Year_Week'] = weekly['Year'].astype(str) + '-W' + weekly['Week'].astype(str).str.zfill(2)
    weekly.sort_values(by=['Year', 'Week'], inplace=True)
    assert not weekly.empty
    assert (weekly['Total Amount (LKR)'] > 0).all()

def test_product_price_analysis(loaded_data):
    analysis = ProductPriceAnalysis(loaded_data)
    # Replicate core logic for testing
    loaded_data_copy = loaded_data.copy() # Work on a copy
    loaded_data_copy['Month'] = loaded_data_copy['Date'].dt.to_period('M')
    price_means = loaded_data_copy.groupby(['Product', 'Month'])['Price (LKR)'].mean().unstack(fill_value=0)
    assert not price_means.empty
    assert (price_means >= 0).all().all() # Check all elements in the DataFrame

def test_product_preference_analysis_overall(loaded_data):
    analysis = ProductPreferenceAnalysis(loaded_data)
    # Replicate core logic for testing
    preference = loaded_data.groupby('Product')['Quantity'].sum().sort_values(ascending=False)
    assert not preference.empty
    assert (preference > 0).all()

def test_product_preference_analysis_by_branch(loaded_data):
    analysis = ProductPreferenceAnalysis(loaded_data)
    # Replicate core logic for testing with branch
    branch = loaded_data['Branch'].unique()[0]
    filtered_data = loaded_data[loaded_data['Branch'] == branch].copy() # Work on a copy
    grouped = filtered_data.groupby('Product')['Quantity'].sum().sort_values(ascending=False)
    assert not grouped.empty
    assert (grouped >= 0).all()

def test_total_sales_distribution_analysis(loaded_data):
    analysis = TotalSalesDistributionAnalysis(loaded_data)
    # Replicate core logic for testing
    distribution = loaded_data.groupby('Branch')['Total Amount (LKR)'].sum()
    assert not distribution.empty
    assert distribution.sum() > 0 # Ensure total sales are positive
    percentage = (distribution / distribution.sum()) * 100
    total = percentage.sum()
    assert 99.9 <= total <= 100.1 # Allow for minor floating point inaccuracies

def test_no_nulls_in_critical_columns(loaded_data):
    for column in ['Date', 'Branch', 'Product', 'Price (LKR)', 'Total Amount (LKR)', 'Quantity']:
        assert loaded_data[column].notnull().all()

def test_date_format_parsed_correctly(loaded_data):
    assert pd.api.types.is_datetime64_any_dtype(loaded_data['Date'])

def test_price_column_is_float(loaded_data):
    assert pd.api.types.is_float_dtype(loaded_data['Price (LKR)'])

def test_quantity_column_is_numeric(loaded_data):
    assert pd.api.types.is_numeric_dtype(loaded_data['Quantity'])
"""

# Save to file
with open("test_sales_dashboard.py", "w") as f:
    f.write(T)

# Then run tests in Colab
!pytest test_sales_dashboard.py

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.4.1, pluggy-1.6.0
rootdir: /content
plugins: anyio-4.9.0, langsmith-0.4.8, typeguard-4.4.4
collected 10 items                                                             

test_sales_dashboard.py ..........                                       [100%]

============================== 10 passed in 3.00s ==============================
